In [1]:
import glob
import os
import pandas as pd

from file_utils import open_json, write_json

In [4]:
# list all csv present in data/ except the ones in other
exceptions = ["others", "PON-All_train_dataset", "O2567_new", "mutation-stability-data", "ProTstab_train_dataset"]
csv_lst = glob.glob("./data/*/*.csv")
print(f"{csv_lst=}")

for path in csv_lst:
    if not any(e in path for e in exceptions):
        df = pd.read_csv(path)
        # print(path, df.columns.to_list())

csv_lst=['./data/O2567_new/O2567_new.csv', './data/mutation-stability-data/tm.csv', './data/mutation-stability-data/test.csv', './data/mutation-stability-data/train.csv', './data/others/amino_acid_translation.csv', './data/ProTstab_train_dataset/protein_ids_training_set.csv', './data/ProTstab_train_dataset/protein_ids_blind_set.csv', './data/FireProtDB/fire_ddg.csv', './data/FireProtDB/fire_dtm.csv', './data/FireProtDB/fire.csv', './data/Ssym+/Data_s669_with_predictions.csv', './data/Ssym+/Ssym+_experimental.csv', './data/Ssym+/Ssym+_rosetta.csv', './data/PON-All_train_dataset/All_species_train.csv', './data/PON-All_train_dataset/Blind_All_species.csv', './data/DeepDDG_train_dataset/datasetDDG_train.csv', './data/Kaggle/updated_train.csv', './data/Kaggle/test.csv']


In [5]:
# load Ssym+ and remove predictions and index columns
ssym_df=pd.read_csv("./data/Ssym+/Data_s669_with_predictions.csv")

ssym_df.drop(columns=ssym_df.columns[10:], axis=1, inplace=True)
ssym_df.drop(columns=ssym_df.columns[0], axis=1, inplace=True)
ssym_df.head()

,Protein,PDB_Mut,Mut_seq,TEMP,pH,DDG_checked_dir,DOI,nmr_xray,resolution
0,1A0FA,S11A,S11A,329.83,6.5,-1.80,10.1042/BJ20061707,EXPDTA X-RAY DIFFRACTION,REMARK 2 RESOLUTION. 2.10 ANGSTROMS.
1,1A7VA,A104H,A104H,298.15,6.5,-2.69,10.1016/j.jmb.2009.07.074,EXPDTA X-RAY DIFFRACTION,REMARK 2 RESOLUTION. 2.30 ANGSTROMS.
2,1A7VA,A66H,A66H,298.15,6.5,-1.98,10.1016/j.jmb.2009.07.074,EXPDTA X-RAY DIFFRACTION,REMARK 2 RESOLUTION. 2.30 ANGSTROMS.
3,1A7VA,A91H,A91H,298.15,6.5,-1.70,10.1016/j.jmb.2009.07.074,EXPDTA X-RAY DIFFRACTION,REMARK 2 RESOLUTION. 2.30 ANGSTROMS.
4,1A7VA,D3H,D3H,298.15,6.5,-1.36,10.1016/j.jmb.2009.07.074,EXPDTA X-RAY DIFFRACTION,REMARK 2 RESOLUTION. 2.30 ANGSTROMS.


In [15]:
thermomut_df = pd.read_json("./data/ThermoMutDB/thermomutdb.json")

columns_to_rm = ["year"]
thermomut_df.drop(columns=columns_to_rm, axis=1, inplace=True)
# print(thermomut_df.columns.to_list())
thermomut_df.head()

,DOI,weight,blosum62,measure,pos,pdbs_template,protein,PDBs_template,id,aro,...,res_depth,temperature,acc,don,dtm,pam250,length,uniprot,PMID,PIR_ID
0,10.1093/oxfordjournals.jbchem.a132423,28726.09,-1.0,CD,0.0,None,Tryptophan synthase alpha chain,NaN,1,0.0,...,3.53,298.95,-2.0,0.0,NaN,0.0,268.0,P0A878,378988.0,TSECA
1,10.1093/oxfordjournals.jbchem.a132423,28723.02,-2.0,CD,0.0,None,Tryptophan synthase alpha chain,NaN,2,0.0,...,3.53,298.95,-1.0,1.0,NaN,-2.0,268.0,P0A878,378988.0,TSECA
2,10.1016/0005-2795(77)90166-0,18611.97,1.0,CD,0.0,None,Endolysin,NaN,3,-3.0,...,3.85,298.15,0.0,0.0,-6.3,1.0,164.0,P00720,911878.0,LZBPT4
3,10.1016/0005-2795(77)90166-0,18798.17,0.0,CD,NaN,None,Endolysin,NaN,4,NaN,...,NaN,298.15,NaN,NaN,-6.2,1.0,164.0,P00720,911878.0,LZBPT4
4,10.1016/0005-2795(77)90166-0,18798.17,1.0,CD,NaN,None,Endolysin,NaN,5,NaN,...,NaN,298.15,NaN,NaN,-6.6,0.0,164.0,P00720,911878.0,LZBPT4
